In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
import scipy

xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

In [2]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
gcs = gcsfs.GCSFileSystem(token='anon')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


In [3]:
source_ls = ["CESM2", "MRI-ESM2-0", "CNRM-CM6-1", "MIROC6", "IPSL-CM6A-LR", "HadGEM3-GC31-LL", "CanESM5", "TaiESM1"]
exp_ls = ["piControl", "abrupt-4xCO2", "amip"]

gcs = gcsfs.GCSFileSystem(token='anon')
for src in range(len(source_ls)):
    for exp in range(len(exp_ls)):
        df_search = df.query("activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'psl'")
        if exp_ls[exp] == "CNRM-CM6-1":
            df_search = df_search.query(f"experiment_id == '{exp_ls[exp]}' & source_id == '{source_ls[src]}' & member_id == 'r1i1p1f2'")
        elif exp_ls[exp] == "CNRM-CM6-1" and source_ls[src] != "piControl": 
            df_search = df_search.query(f"experiment_id == '{exp_ls[exp]}' & source_id == '{source_ls[src]}' & member_id == 'r1i1p1f3'")
        else:
            df_search = df_search.query(f"experiment_id == '{exp_ls[exp]}' & source_id == '{source_ls[src]}' & member_id == 'r1i1p1f1'")
        if len(df_search) != 1:
            f = open("missing_file.txt", "a")
            if len(df_search) == 0:
                f.write(f"\nsource: {source_ls[src]}, exp: {exp_ls[exp]}\n")
            else:
                f.write(f"\nsource: {source_ls[src]}, exp: {exp_ls[exp]}, more than 1 query is detected\n")
                f.write(df_search)
            f.close()
        else:
            ds = xr.open_zarr(gcs.get_mapper(df_search.zstore.values[0]), consolidated=True)

/data/keeling/a/kchoo3/anaconda3/envs/myenv/lib/python3.12/site-packages/xarray/coding/times.py:995: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/data/keeling/a/kchoo3/anaconda3/envs/myenv/lib/python3.12/site-packages/xarray/core/indexing.py:630: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  array = array.get_duck_array()
/data/keeling/a/kchoo3/anaconda3/envs/myenv/lib/python3.12/site-packages/xarray/core/indexing.py:500: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(self.get_duck_array(), dtype=dtype)
/data/keeling/a/kchoo3/anaconda3/envs/myenv/

In [9]:
#df_something = df.query("activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'psl' & member_id == 'r1i1p1f1' & experiment_id == 'piControl' & source_id == 'CESM2'")
df_something = df.query("activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'psl' & member_id == 'r1i1p1f1'")
ds = xr.open_zarr(gcs.get_mapper(df_something.zstore.values[0]), consolidated=True)
ds

<xarray.Dataset> Size: 1GB
Dimensions:    (bnds: 2, lat: 180, lon: 288, time: 6000)
Coordinates:
  * bnds       (bnds) float64 16B 1.0 2.0
  * lat        (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bnds   (lat, bnds) float64 3kB dask.array<chunksize=(180, 2), meta=np.ndarray>
  * lon        (lon) float64 2kB 0.625 1.875 3.125 4.375 ... 356.9 358.1 359.4
    lon_bnds   (lon, bnds) float64 5kB dask.array<chunksize=(288, 2), meta=np.ndarray>
  * time       (time) object 48kB 0001-01-16 12:00:00 ... 0500-12-16 12:00:00
    time_bnds  (time, bnds) object 96kB dask.array<chunksize=(6000, 2), meta=np.ndarray>
Data variables:
    psl        (time, lat, lon) float32 1GB dask.array<chunksize=(942, 180, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          Coupled climate state after 400 years of spinup, ...
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    comment:                <null ref>
    ...                     ...
    variable_id:            psl
    variant_info:           N/A
    variant_label:          r1i1p1f1
    status:                 2019-11-09;created;by nhn2@columbia.edu
    netcdf_tracking_ids:    hdl:21.14100/1a678f3c-4951-46ed-84e5-c6a1763ab08d...
    version_id:             v20180701

In [ ]:
ds.to_netcdf()

In [10]:
new_ds = ds
new_ds = new_ds.drop_vars('lat_bnds')
new_ds = new_ds.drop_vars('lon_bnds')
new_ds = new_ds.drop_vars('time_bnds')
new_ds

<xarray.Dataset> Size: 1GB
Dimensions:  (bnds: 2, lat: 180, lon: 288, time: 6000)
Coordinates:
  * bnds     (bnds) float64 16B 1.0 2.0
  * lat      (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 2kB 0.625 1.875 3.125 4.375 ... 356.9 358.1 359.4
  * time     (time) object 48kB 0001-01-16 12:00:00 ... 0500-12-16 12:00:00
Data variables:
    psl      (time, lat, lon) float32 1GB dask.array<chunksize=(942, 180, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          Coupled climate state after 400 years of spinup, ...
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    comment:                <null ref>
    ...                     ...
    variable_id:            psl
    variant_info:           N/A
    variant_label:          r1i1p1f1
    status:                 2019-11-09;created;by nhn2@columbia.edu
    netcdf_tracking_ids:    hdl:21.14100/1a678f3c-4951-46ed-84e5-c6a1763ab08d...
    version_id:             v20180701

In [ ]:
new_ds.to_netcdf()